### Первичные параметры
1. $R_0$, [Ом/м, Ом/км] – активное сопротивление прямого и обратного проводов линии (жилы и оболочки) на единицу длины;
2. $L_0$, [Гн/м, Гн/км] – индуктивность петли, образуемой прямым и обратным проводами;
3. $C_0$, [Ф/м, Ф/км] – емкость между проводами (емкость кабеля);
4. $G_0$, [См/м, См/км] – проводимость между проводами - жилой и оболочкой.

### Формулы для расчёта:
Распределение токов и потенциала в длинной линии при не зависящих от пространственной координаты первичных параметрах может быть рассчитано по следующим формулам (при подключении источника ЭДС):<br><br>
<div style="font-size: 20px;">
$U(x) = Ae^{\gamma*x} + Be^{-\gamma*x}$,<br><br>
$I(x) = \frac{1}{Z_0}(-Ae^{\gamma*x} + Be^{-\gamma*x})$,<br><br>
$\gamma = \sqrt{(R + j\omega*L) * (G + j\omega*C)}$,<br><br>
$Z_0 = \sqrt{\frac{(R + j\omega*L)}{(G + j\omega*C)}}$,  $Y = \frac{Z_н}{Z_0}$,<br><br>
$A = \frac{E}{1 + e^{2\gamma*l}\frac{Y + 1}{Y - 1}}$,  $B = \frac{Ee^{2\gamma*l}\frac{Y + 1}{Y - 1}}{1 + e^{2\gamma*l}\frac{Y + 1}{Y - 1}}$,<br><br>
$P(x) = \frac{|Be^{-\gamma*x}|^2}{Re(Z_0)} - \frac{|Ae^{\gamma*x}|^2}{Re(Z_0)}, P = U * I$
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import widgets, Layout


def get_A_B(E, gamma, l, Y):
    A = E / (1 + np.exp(2*gamma*l)*(Y+1)/(Y-1))
    B = (E*np.exp(2*gamma*l)*(Y+1)/(Y-1)) / (1 + np.exp(2*gamma*l)*(Y+1)/(Y-1))
    return A, B
    
    
def get_gamma_z0(R, L, C, G, w):
    gamma = np.sqrt(complex(R, w*L) * complex(G, w*C))
    z0 = np.sqrt(complex(R, w*L) / complex(G, w*C))
    return gamma, z0


def get_U_I(x, A, B, gamma, z0):
    U_x = A*np.exp(gamma*x) + B*np.exp(-gamma*x)
    I_x = (-A*np.exp(gamma*x) + B*np.exp(-gamma*x)) / z0
    return U_x, I_x


def get_P_rn(A, B, gamma, z0, l):
    Rez0 = z0.real
    Prn = abs(((B*np.exp(-gamma*l))**2)) / Rez0 - abs(((A*np.exp(gamma*l))**2)) / Rez0
    return Prn


def draw_U_I_x(x, U_x, I_x):
    fig = plt.figure(figsize=(17, 10))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    ax1.set_title('Зависимость U(x)')
    ax1.set_xlabel('x, км')
    ax1.set_ylabel('U(x), В')
    ax2.set_title('Зависимость I(x)')
    ax2.set_xlabel('x, км')
    ax2.set_ylabel('I(x), А')
    ax1.plot(x, U_x.real, label='Real')
    ax1.plot(x, U_x.imag, label='Imag')
    ax2.plot(x, I_x.real, label='Real')
    ax2.plot(x, I_x.imag, label='Imag')
    ax1.legend(loc='best')
    ax2.legend(loc='best')
    plt.tight_layout()


def draw_P(E, R, L, C, G, rn, l, w_P):
    w = np.linspace(2*np.pi*w_P[0]*10**3, 2*np.pi*w_P[1]*10**3, 1000)
    fig = plt.figure(figsize=(17, 10))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    P_list = []
    for ww in w:
        gamma, z0 = get_gamma_z0(R, L, C, G, ww)
        Y = rn / z0
        A, B = get_A_B(E, gamma, l, Y)
        U_x, I_x = get_U_I(l, A, B, gamma, z0)
        #P = get_P_rn(A, B, gamma, z0, l)
        P = U_x*complex(I_x.real, -I_x.imag)
        P_list.append(P)
    for ax in (ax1, ax2):
        ax.set_xlabel('ω, Гц')
        ax.set_ylabel('P(ω), Вт')
        ax.set_title('Зависимость мощности на нагрузке от частоты')
    ax1.plot(w, np.array(P_list).real, label='Real')
    ax2.plot(w, np.array(P_list).imag, label='Imag')
    ax1.legend()
    ax2.legend()
    plt.tight_layout()
        

def main(*args):
    R = w_R.value
    L = w_L.value*10**-3
    C = w_C.value*10**-9
    G = w_G.value*10**-6
    E = w_E.value*10**3
    rn = w_rn.value
    l = w_l.value
    w = 2*np.pi*w_w.value*10**3
    w_P = w_4P.value
    with out2:
        clear_output(wait=True)
        x = np.linspace(0, l, 1000)
        gamma, z0 = get_gamma_z0(R, L, C, G, w)
        beta = gamma.imag
        Y = rn / z0
        A, B = get_A_B(E, gamma, l, Y)
        print('Длинна волны:', (2*np.pi) / beta)
        U_x, I_x = get_U_I(x, A, B, gamma, z0)
        plt.rcParams['font.size'] = '20'
        draw_U_I_x(x, U_x, I_x)
        draw_P(E, R, L, C, G, rn, l, w_P)
        plt.show()
        
        
        
box_layout = Layout(display='flex',
                    justify_content='space-between',
                    width='60%')
out1 = widgets.Output()
out2 = widgets.Output()
w_rn_L = widgets.Label('Нагрузочное сопротивление, Rn [Ом]')
w_rn = widgets.FloatSlider(description='', min=0, max=1000, step=1, value=100)
w_l_L = widgets.Label('Длина, l [км]')
w_l = widgets.FloatSlider(description='', min=0, max=100, step=1, value=15)
w_E_L = widgets.Label('ЭДС, E [кВ]')
w_E = widgets.FloatSlider(description='', min=0, max=100, step=0.1, value=10)
w_w_L = widgets.Label('Частота, ω [кГц]')
w_w = widgets.FloatSlider(description='', min=0, max=1000, step=1, value=500)
w_desc_L = widgets.Label('Первичные параметры:')
w_R_L = widgets.Label('Сопротивление, R [Ом/км]')
w_R = widgets.FloatSlider(description='', min=0, max=100, step=1, value=20)
w_L_L = widgets.Label('Индуктивность, L [мГн/км]')
w_L = widgets.FloatSlider(description='', min=0, max=2, step=0.01, value=0.25)
w_C_L = widgets.Label('Емкость, C  [нФ/км]')
w_C = widgets.FloatSlider(description='', min=0, max=100, step=0.1, value=44.5)
w_G_L = widgets.Label('Проводимость, G [мкСм/км]')
w_G = widgets.FloatSlider(description='', min=0, max=20, step=0.1, value=3)
w_4P_L = widgets.Label('Диапазон частот для исследования мощности на нагрузке [кГц]')
w_4P = widgets.FloatRangeSlider(min=20, max=1000, step=1, value=[450, 550])
button = widgets.Button(description='Запустить', button_style='primary')
button.on_click(main)
display(widgets.VBox([out1, out2]))
with out1:
    display(widgets.HBox([w_rn_L, w_rn], layout=box_layout),
            widgets.HBox([w_l_L, w_l], layout=box_layout),
            widgets.HBox([w_E_L, w_E], layout=box_layout),
            widgets.HBox([w_w_L, w_w], layout=box_layout),
            w_desc_L,
            widgets.HBox([w_R_L, w_R], layout=box_layout),
            widgets.HBox([w_L_L, w_L], layout=box_layout),
            widgets.HBox([w_C_L, w_C], layout=box_layout),
            widgets.HBox([w_G_L, w_G], layout=box_layout),
            widgets.VBox([w_4P_L, w_4P], layout=box_layout),
            button
            )

ModuleNotFoundError: No module named 'ipywidgets'